In [1]:
import torch
import torch.nn as nn
import numpy as np
from MemSE.train_test_loop import accuracy
from MemSE.utils import count_parameters
from MemSE.model_load import load_model
from MemSE.dataset import get_dataloader
from MemSE.network_manipulations import conv_to_fc
from MemSE.MemSE import MemSE
from MemSE.MemristorQuant import MemristorQuant

device = torch.device('cpu')

train_loader, valid_loader, test_loader, nclasses, input_shape = get_dataloader('CIFAR10')
criterion = nn.CrossEntropyLoss().to(device)
model = load_model('smallest_vgg', nclasses).to(device)
print(count_parameters(model))
model = conv_to_fc(model, input_shape).to(device)
print(count_parameters(model))
batch, target = next(iter(valid_loader))
batch, target = batch.to(device), target.to(device)
original_output = model(batch)
original_acc = accuracy(original_output, target)
print(f'Original acc is {original_acc}')

print('Before quant')
for name, val in model.named_parameters():
    print(name)
    print(val)
print('8'*5)

quanter = MemristorQuant(model)
quanter.quant()
for name, val in model.named_parameters():
    print(name)
    print(val)
quanted_output = model(batch)
quanted_acc = accuracy(quanted_output, target)
print(f'Quanted acc is {quanted_acc}')

[pyKeOps]: Warning, no cuda detected. Switching to cpu only.
Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified
Loading model checkpoint
4030
8061088
Original acc is [tensor(60.9375)]
features.0.2.weight
Parameter containing:
tensor([[-0.5927, -0.7306, -1.3765,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000, -0.5927, -0.7306,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000, -0.5927,  ...,  0.0000,  0.0000,  0.0000],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.5485,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  1.0296,  0.5485,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.9751,  1.0296,  0.5485]],
       requires_grad=True)
features.3.2.weight
Parameter containing:
tensor([[ 0.1329,  2.7527, -1.1311,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.1329,  2.7527,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.1329,  ...,  0.0000,  0.0000,  0.0000]

In [ ]:
memse = MemSE(model, quanter)
x, gamma, P_tot, mses = memse.mse_forward(batch)